<a href="https://colab.research.google.com/github/teerasitk/EEMATH/blob/main/ErrorCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def compSyndrone(decision:np.ndarray, H:np.ndarray)->np.ndarray:
  """
  decision: 7xn binary array
  H: hamming decoder matrix of size 7 x 3
  return syndrone binary array of size 3 x N
  """
  s = (np.dot(H.T, decision) % 2).astype('int')
  return s

# error_pattern = [[0, 0, 0, 0, 0, 0, 0, 0], # 000
#                  [0, 0, 0, 0, 0, 0, 0, 1], # 001
#                  [0, 0, 0, 0, 0, 0, 1, 0], # 010
#                  [0, 0, 1, 0, 0, 0, 0, 0], # 011
#                  [0, 0, 0, 0, 0, 1, 0, 0], # 100
#                  [0, 1, 0, 0, 0, 0, 0, 0], # 101
#                  [1, 0, 0, 0, 0, 0, 0, 0], # 110
#                  [0, 0, 0, 1, 0, 0, 0, 0]] # 111

def errorCorrection(decision, H, error_pattern):
  syndrome = compSyndrone(decision, H).astype('int')
  value = syndrome[0, :] *4 + syndrome[1, :] * 2  + syndrome[2, :]
  error_loc = error_pattern[value].T
  corrected_decision = (decision + error_loc) %2
  return corrected_decision

In [3]:
def hammingEncoder(src: np.ndarray) -> np.ndarray:
  """
  src: 4 x N binary array
  return: 7 x n binary array
  """
  A = np.array([[1, 1, 0],
                [1, 0, 1],
                [0, 1, 1],
                [1, 1, 1]])
  I4 = np.eye(4)
  G = np.concatenate(([I4, A.T]), axis=0) # Encoder Matrix
  hamming_src = np.dot(G, src) % 2 # modular 2 to make sure that the output is binary
  return hamming_src.astype('int')


def hammingDecoder(src: np.ndarray) -> np.ndarray:
  """
  src: 7 x N binary array
  return: 4 x n binary array
  """
  A = np.array([[1, 1, 0],
              [1, 0, 1],
              [0, 1, 1],
              [1, 1, 1]])
  I3 = np.eye(3)
  H = np.concatenate(([A, I3]), axis=0) # Decoder Matrinx
  error_pattern = [np.zeros(7,) for k in range(8)] # extract Syndrome
  a = np.zeros((7,)) # no error first
  s = np.dot(H.T,a).astype('int')
  error_pattern[s[0]*4 + s[1] *2 + s[2]] = a
  for k in range(7):
    a = np.zeros((7,))
    a[k] = 1
    s = np.dot(H.T,a).astype('int')
    error_pattern[s[0] * 4 + s[1] * 2 + s[2]] = a
  error_pattern = np.array(error_pattern)
  corrected_codes =  errorCorrection(src, H, error_pattern)
  return corrected_codes[:4, :].astype('int')


In [4]:
# parameter for experimental setup
N = 8# number samples
signal_power  = 1
SNR_dB = 0 # 10 * np.log10(SNR)
SNR = 10 ** (SNR_dB/10)
noise_power = signal_power / SNR
noise_magnitude = np.sqrt(noise_power)
print(f"Signal to noise ration: {SNR_dB:0.2f} dB with noise magnitude: {noise_magnitude:0.3f}")

Signal to noise ration: 0.00 dB with noise magnitude: 1.000


In [12]:
# source encoder
src = np.random.choice(2, size=(4, N)) # 8 binary codes of size 4 bits
hamming_src = hammingEncoder(src)
for k in range(N):
  print("Input Binary 4 bits: ", src[:, k], "Hamming Code 7 bits: ", hamming_src[:, k])


Input Binary 4 bits:  [1 0 0 0] Hamming Code 7 bits:  [1 0 0 0 1 1 0]
Input Binary 4 bits:  [1 1 0 1] Hamming Code 7 bits:  [1 1 0 1 1 0 0]
Input Binary 4 bits:  [1 1 0 1] Hamming Code 7 bits:  [1 1 0 1 1 0 0]
Input Binary 4 bits:  [0 0 1 1] Hamming Code 7 bits:  [0 0 1 1 1 0 0]
Input Binary 4 bits:  [1 0 1 1] Hamming Code 7 bits:  [1 0 1 1 0 1 0]
Input Binary 4 bits:  [0 0 1 0] Hamming Code 7 bits:  [0 0 1 0 0 1 1]
Input Binary 4 bits:  [0 0 0 1] Hamming Code 7 bits:  [0 0 0 1 1 1 1]
Input Binary 4 bits:  [1 0 0 0] Hamming Code 7 bits:  [1 0 0 0 1 1 0]


In [13]:
# convert src into signal to send "1" we use 1 volt to send "0" we use -1 volt.
signal = hamming_src  * 2 - 1
for k in range(N):
  print("Hamming Code 7 bits: ", hamming_src[:, k], "Signal : ", signal[:, k])

Hamming Code 7 bits:  [1 0 0 0 1 1 0] Signal :  [ 1 -1 -1 -1  1  1 -1]
Hamming Code 7 bits:  [1 1 0 1 1 0 0] Signal :  [ 1  1 -1  1  1 -1 -1]
Hamming Code 7 bits:  [1 1 0 1 1 0 0] Signal :  [ 1  1 -1  1  1 -1 -1]
Hamming Code 7 bits:  [0 0 1 1 1 0 0] Signal :  [-1 -1  1  1  1 -1 -1]
Hamming Code 7 bits:  [1 0 1 1 0 1 0] Signal :  [ 1 -1  1  1 -1  1 -1]
Hamming Code 7 bits:  [0 0 1 0 0 1 1] Signal :  [-1 -1  1 -1 -1  1  1]
Hamming Code 7 bits:  [0 0 0 1 1 1 1] Signal :  [-1 -1 -1  1  1  1  1]
Hamming Code 7 bits:  [1 0 0 0 1 1 0] Signal :  [ 1 -1 -1 -1  1  1 -1]


In [14]:
# All transmission in communication system is noisy,
received_signal = signal + noise_magnitude * np.random.randn(7, 8) # 7 bits 8 codes
for k in range(N):
  print("Signal : ", signal[:, k], "Recieved Signal: ", received_signal[:, k] )

Signal :  [ 1 -1 -1 -1  1  1 -1] Recieved Signal:  [ 2.33213307 -1.61903969 -1.80153348 -1.10228016  1.09372801  0.59923692
 -0.11173786]
Signal :  [ 1  1 -1  1  1 -1 -1] Recieved Signal:  [ 1.35356663  0.10625971 -2.6051937   2.74864929  1.45934266 -2.22794164
  0.18109885]
Signal :  [ 1  1 -1  1  1 -1 -1] Recieved Signal:  [ 0.11638774  1.41383289 -1.89456801  1.42915842  1.44598351  0.32363364
 -1.4594616 ]
Signal :  [-1 -1  1  1  1 -1 -1] Recieved Signal:  [ 0.05278679 -0.03084995  2.83189398  1.52479967  0.50503073 -0.663081
 -1.07051109]
Signal :  [ 1 -1  1  1 -1  1 -1] Recieved Signal:  [ 1.09223951 -0.1347161  -0.00944854  1.27247783 -0.78345605  0.89395698
 -2.62961757]
Signal :  [-1 -1  1 -1 -1  1  1] Recieved Signal:  [-2.18704685 -2.40959132 -0.28259549 -0.42396893  1.05346673 -0.06541594
  1.22034749]
Signal :  [-1 -1 -1  1  1  1  1] Recieved Signal:  [-0.25065108 -2.75212446 -0.97115295  1.91003444  0.70970647  0.7682322
 -0.19180197]
Signal :  [ 1 -1 -1 -1  1  1 -1] Reci

In [15]:
# make binary decision of recieved signal >= 0, we decide "1" and if signal <0, we decide "1"
decided_code = (received_signal >=0).astype('int')
for k in range(N):
  print("Hamming Code 7 bits: ", hamming_src[:, k], "decided code: ", decided_code[:, k] )
num_bit_errors = (hamming_src != decided_code).sum()
error_rate = (hamming_src != decided_code).mean()
print(f"There are {num_bit_errors} from {N*7} bits with error rate {error_rate*100: 0.3f}%")
# 4 bit data
num_bit_errors = (hamming_src[:4, :] != decided_code[:4, :]).sum()
error_rate = (hamming_src[:4, :] != decided_code[:4, :]).mean()
print(f"For the first 4 bit data. There are {num_bit_errors} from {N*4} bits with error rate {error_rate*100: 0.3f}%")

Hamming Code 7 bits:  [1 0 0 0 1 1 0] decided code:  [1 0 0 0 1 1 0]
Hamming Code 7 bits:  [1 1 0 1 1 0 0] decided code:  [1 1 0 1 1 0 1]
Hamming Code 7 bits:  [1 1 0 1 1 0 0] decided code:  [1 1 0 1 1 1 0]
Hamming Code 7 bits:  [0 0 1 1 1 0 0] decided code:  [1 0 1 1 1 0 0]
Hamming Code 7 bits:  [1 0 1 1 0 1 0] decided code:  [1 0 0 1 0 1 0]
Hamming Code 7 bits:  [0 0 1 0 0 1 1] decided code:  [0 0 0 0 1 0 1]
Hamming Code 7 bits:  [0 0 0 1 1 1 1] decided code:  [0 0 0 1 1 1 0]
Hamming Code 7 bits:  [1 0 0 0 1 1 0] decided code:  [1 0 0 0 1 1 0]
There are 8 from 56 bits with error rate  14.286%
For the first 4 bit data. There are 3 from 32 bits with error rate  9.375%


In [16]:
# Using Hamming decoder
hamming_decoded_code = hammingDecoder(decided_code)
for k in range(N):
  print("source Code 4 bits: ", src[:, k], "hamming decoded code: ", hamming_decoded_code[:, k] )
num_bit_errors = (src !=hamming_decoded_code).sum()
error_rate = (src != hamming_decoded_code).mean()
print(f"For the first 4 bit data with Hamming Decoder.")
print(f"There are {num_bit_errors} from {N*4} bits with error rate {error_rate*100: 0.3f}%")

source Code 4 bits:  [1 0 0 0] hamming decoded code:  [1 0 0 0]
source Code 4 bits:  [1 1 0 1] hamming decoded code:  [1 1 0 1]
source Code 4 bits:  [1 1 0 1] hamming decoded code:  [1 1 0 1]
source Code 4 bits:  [0 0 1 1] hamming decoded code:  [0 0 1 1]
source Code 4 bits:  [1 0 1 1] hamming decoded code:  [1 0 1 1]
source Code 4 bits:  [0 0 1 0] hamming decoded code:  [0 1 0 0]
source Code 4 bits:  [0 0 0 1] hamming decoded code:  [0 0 0 1]
source Code 4 bits:  [1 0 0 0] hamming decoded code:  [1 0 0 0]
For the first 4 bit data with Hamming Decoder.
There are 2 from 32 bits with error rate  6.250%
